In [1]:
using SparseArrays
using DelimitedFiles
using LinearAlgebra
using IterativeSolvers

In [2]:
# domínio
a = -2.0
b = 2.0
;

In [3]:
k = 1 # grau do polinômio de interpolação
nen = k + 1 # número de nós do elemento
nint = k + 1 # número de pontos de integração
;

In [5]:
# vetores e variáveis auxiliares
max_degree = 6
error = zeros(max_degree)
h_s = zeros(max_degree)
;

In [6]:
function f(x)
    angle_in_radians = (π/1) * x
    return sin(angle_in_radians)*sin(angle_in_radians)
end
;

In [8]:
function shl(nen, nint)
    if nint == 2
        pt = zeros(2)
        w = pt = zeros(2)

        pt[1] = -sqrt(3.0)/3.0
        pt[1] = sqrt(3.0)/3.0

        w[1] = 1.0
        w[1] = 1.0
        sh = zeros(2, nint)
    end

    for l in 1:nint
        t = pt[l]
        if nen == 2
            sh[1,l] = (1.0-t)/2.0;
            sh[2,l] = (1.0+t)/2.0;
        end
    end

    return sh
end
;

In [9]:
function we(nint)
    w = zeros(2)
    if nint == 2
        w[1] = 1.0
        w[2] = 1.0
    end
    return w
end
;

In [10]:
pos = 1
for degree in 2:max_degree
    nel = 4^(degree)
    h = (b-a)/nel

    np = k * nel + 1
    
    xl = zeros(np)
    
    xl[1] = a
    for i in 2:np
        xl[i] = xl[i-1] + h
    end
    
    # matrizes globais
    M = zeros(np, np)
    F = zeros(np)
    
    for n in 1:nel
        Me = zeros(np, np)
        Fe = zeros(np)
        shg = shl(nen, nint)
        w = we(nint)
    
        for l in 1:nint
            xx = 0
            for i in 1:nen
                xx = xx + shg[i,l]*xl[n+i-1]
            end
            for j in 1:nen
                Fe[j] = Fe[j] + f(xx) * shg[j,l] * w[l] * (h/2.0)
                for i in 1:nen
                    Me[i,j] = Me[i,j] + shg[i,l] * shg[j,l] * w[l] * (h/2.0)
                end
            end
        end
        for j in 1:nen
            F[n+j-1] = F[n+j-1] + Fe[j]
            for i in 1:nen
                M[n+i-1, n+j-1] = M[n+i-1, n+j-1] + Me[i,j]
            end
        end
    end
    # alpha = zeros(np)
    # println(M)
    # println(F)
    inital_guess = zeros(np)
    alpha = zeros(np)
    alpha = cg!(inital_guess, M, F)
    name_xl = "resultados_xl_$(nel).dat"
    name_alphas = "resultados_alphas_$(nel).dat"
    writedlm(name_xl, xl)
    writedlm(name_alphas, alpha)

    erul2 = 0

    for n in 1:nel
        eru = 0
        shg = shl(nen, nint)
        w = we(nint)

        for l in 1:nint
            uh = 0.0
            xx = 0.0

            for i in 1:nen
                uh = uh + shg[i,l]*alpha[n+i-1]
                xx = xx + shg[i,l]*xl[n+i-1]
            end
            eru = eru + ((f(xx)-uh)^2)*w[l]*(h/2)
        end
        erul2 = erul2 + eru
    end
    erul2 = sqrt(erul2)
    
    h_s[pos] = h
    error[pos] = erul2 
    pos = pos + 1
    writedlm("matrix_m_$(nel).dat", M)
    writedlm("matrix_f_$(nel).dat", F)
end

writedlm("erros.dat", error)
writedlm("h_s.dat", h_s)
;

In [11]:
using WAV
y, fs = wavread(raw"/home/joao-oliveira/Downloads/mixkit-alert-alarm-1005.wav")
wavplay(y, fs)